##### Concatenate North American(exclude Alaska) and Mexican BumbleBee data

Combining tables in SAS is like worker bees stacking honeycombs—each dataset adding its own golden sweetness. With SET statements or PROC APPEND, you can seamlessly merge observations into a thriving hive of insights. Let’s get buzzing! 🐝

Know thy data using PROC CONTENTS. Before concatenating tables, first ensure that both datasets have the same structure (i.e., same variable names and types).

In [1]:
proc contents data=dst1;
run;
proc contents data=dst2;
run;

SyntaxError: invalid syntax (2613055050.py, line 1)

Concatenate the North American and Mexican Bumblebee data using the SET statement

In [ ]:
data dsconc;
set dst1 dst2;
run;

Why did the concatenation fail?

During the Access phase, we used PROC IMPORT to convert CSVs into SAS datasets. By default, PROC IMPORT "guesses" the data structure by examining the first 20 rows to determine variable types and lengths. It assigns the most prevalent data type (numeric or character) to each column. If most of the first 20 rows are missing, SAS defaults to the character data type, and any subsequent numeric values are set to missing. This is why the column occurenceID defined as Character in the Mexican bumblebee data conflicts with the numeric OccurenceId in the North American Bumblebee dataset.   The DATA step, however, offers more control, granularity, and precision for importing data

In [ ]:
ata dst1;
infile "/workspaces/myfolder/SASPythonDataScientists/pattern_decline_N_American_Bumblebees.csv"  dsd firstobs=2;
input 
id 
institutionCode : $8. 
collectionCode : $4. 
basisOfRecord : $17.  
occurrenceID :$9.
catalognumber: $12. 
recordedBy $
year
month 
day 
country :$6. 
stateProvince  : $16.
county : $17.
locality  : $37. 
verbatimLatitude  
verbatimLongitude 
identifiedBy : $18.
scientificName  : $20. 
kingdom  : $8. 
phylum  : $10. 
class : $7. 
order  : $11. 
family  : $6. 
genus :  $6. 
specificEpithet : $13. 
scientificNameAuthorship : $13. 
;

run;


data dst2;
infile "/workspaces/myfolder/SASPythonDataScientists/pattern_decline_Mexican_Bumblebees.csv"  dsd firstobs=2;
input 
id 
institutionCode : $8. 
collectionCode : $4. 
basisOfRecord : $17.  
occurrenceID :$9.
catalognumber: $12. 
recordedBy $
year
month 
day 
country :$6. 
stateProvince  : $16.
county : $17.
locality  : $37. 
verbatimLatitude  
verbatimLongitude 
identifiedBy : $18.
scientificName  : $20. 
kingdom  : $8. 
phylum  : $10. 
class : $7. 
order  : $11. 
family  : $6. 
genus :  $6. 
specificEpithet : $13. 
scientificNameAuthorship : $13. 
;
run;

Now lets see the metadata of the 2 tables, and pay careful attention to the offending type mismatch of the column OccurenceId

In [ ]:
proc contents data=dst1 varnum;
run;
proc contents data=dst2 varnum;
run;

Let's now concatenate the 2 tables (filtering out Alaska from the North American Dataset)

In [ ]:
data dsconc;
set dst1(where=(country <> 'Alaska')) dst2;
run;

🐝 Merging Datasets: A Bumble Bee Bonanza! 🐝

Just like bees waggle, not recite Latin, our data needs common names, not just Bombus jargon. So, we’ll merge our North American & Mexican dataset with a lookup table—letting SAS do the heavy lifting while our data buzzes with clarity! 🍯✨ 

The term “bumblebee” traces its roots to the word bumble, meaning to hum or buzz — a fitting nod to the sound these vital pollinators make. Their scientific name, Bombus, was introduced in 1802 and is derived from Latin and Greek words for a buzzing sound.

In [ ]:
proc contents data=dsconc;
run;

In [ ]:
proc contents data=dst3;
run;

In [ ]:
data dsmerge;
    merge dsconc dst3;
    by scientificname;
run;

prepping the datasets for merge by running a PROC SORT

In [ ]:
proc sort data=dsconc;
    by scientificname;
run;
proc sort data=dst3;
by scientificname;
run;

In [ ]:
data dsmerge;
    merge dsconc(in=inc) dst3(in=ind);
    by scientificname;
    if inc and ind;
run;

In [ ]:
proc contents data=dsmerge;
run;